In [217]:
#import the libraries

import pandas as pd
import numpy as np
import nltk, re, string
from textblob import TextBlob
import emoji
from nltk.tokenize import word_tokenize
import spacy
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

For the test I have used IMDB dataset which I have taken from Kaggle.

In [2]:
#load the data into Dataframe
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

## LowerCasing

In [19]:
#define a function to convert the text into lowercase

def lower_case(text):
    txt_lower = []
    for txt in text:
        txt_lower.append(txt.lower())
    return "".join(txt_lower)

#another way to convert it to lowercase
# df['review'] = df['review'].str.lower()

In [20]:
lower_case('I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy.')

'i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy.'

In [23]:
df['review'] = df['review'].apply(lower_case)

In [24]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


Now we can see all the text are changed to lowercase.

## Remove HTML tags

In [27]:
# define a function to remove HTML tags

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [34]:
df['review'][3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [35]:
df['review'] = df['review'].apply(remove_html)

In [36]:
df['review'][3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

After applying the function we can see all the HTML tags are removed now.

## Remove the URLs

In [47]:
# define a function which will remove the URLs

def remove_urls(text):
    pattern = re.compile(r'https?:\/\/\S+|www\.\S+')
    return pattern.sub(r'',text)

In [49]:
df['review'] = df['review'].apply(remove_urls)

In [51]:
df['review'][3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

## Remove Punctuations

In [55]:
exclude = string.punctuation

In [56]:
# define a function to remove the punctuation from the data

def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [60]:
df['review'] = df['review'].apply(remove_punc)

In [61]:
df['review'][3]

'basically theres a family where a little boy jake thinks theres a zombie in his closet  his parents are fighting all the timethis movie is slower than a soap opera and suddenly jake decides to become rambo and kill the zombieok first of all when youre going to make a film you must decide if its a thriller or a drama as a drama the movie is watchable parents are divorcing  arguing like in real life and then we have jake with his closet which totally ruins all the film i expected to see a boogeyman similar movie and instead i watched a drama with some meaningless thriller spots3 out of 10 just for the well playing parents  descent dialogs as for the shots with jake just ignore them'

## ChatWords Removal

In [108]:
chat_words = {'AFAIK' :'As Far As I Know',
'AFK':'Away From Keyboard',
'ASAP':'As Soon As Possible',
'ATK':'At The Keyboard',
'ATM':'At The Moment',
'A3':'Anytime, Anywhere, Anyplace',
'BAK':'Back At Keyboard',
'BBL':'Be Back Later',
'BBS':'Be Back Soon',
'BFN':'Bye For Now',
'B4N':'Bye For Now',
'BRB':'Be Right Back',
'BRT':'Be Right There',
'BTW':'By The Way',
'B4':'Before',
'B4N':'Bye For Now',
'CU':'See You',
'CUL8R':'See You Later',
'CYA':'See You',
'FAQ':'Frequently Asked Questions',
'FC':'Fingers Crossed',
'FWIW':"For What It's Worth",
'FYI':'For Your Information',
'GAL':'Get A Life',
'GG':'Good Game',
'GN':'Good Night',
'GMTA':'Great Minds Think Alike',
'GR8':'Great!',
'G9':'Genius',
'IC':'I See',
'ICQ':'I Seek you (also a chat program)',
'ILU':'ILU: I Love You',
'IMHO':'In My Honest/Humble Opinion',
'IMO':'In My Opinion',
'IOW':'In Other Words',
'IRL':'In Real Life',
'KISS':'Keep It Simple, Stupid',
'LDR':'Long Distance Relationship',
'LMAO':'Laugh My A.. Off',
'LOL':'Laughing Out Loud',
'LTNS':'Long Time No See',
'L8R':'Later',
'MTE':'My Thoughts Exactly',
'M8':'Mate',
'NRN':'No Reply Necessary',
'OIC':'Oh I See',
'PITA':'Pain In The A..',
'PRT':'Party',
'PRW':'Parents Are Watching',
'QPSA?': '	Que Pasa?',
'ROFL':'Rolling On The Floor Laughing',
'ROFLOL':'Rolling On The Floor Laughing Out Loud',
'ROTFLMAO':'Rolling On The Floor Laughing My A.. Off',
'SK8':'Skate',
'STATS':'Your sex and age',
'ASL':'Age, Sex, Location',
'THX':'Thank You',
'TTFN':'Ta-Ta For Now!',
'TTYL':'Talk To You Later',
'U':'You',
'U2':'You Too',
'U4E':'Yours For Ever',
'WB':'Welcome Back',
'WTF':'What The F...',
'WTG':'Way To Go!',
'WUF':'Where Are You From?',
'W8':'Wait...',
'7K':'Sick:-D Laugher',
'TC': 'Take Care'}

In [109]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': "For What It's Worth",
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great!',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you (also a chat program)',
 'ILU': 'ILU: I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple, Stupid',
 'LDR': 'Long Distance Relationship',
 'LM

In [87]:
# define a function for chatwords removal

def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [114]:
chat_conversion('let\'s meet on Sunday. gn tc')

"let's meet on Sunday. Good Night Take Care"

## Spell check

In [147]:
txt_blob = TextBlob('this is a vry guud seris. I lve to wacth it')

In [148]:
txt_blob.correct()

TextBlob("this is a very good series. I love to watch it")

## StopWords Removal

In [154]:
stp_words = nltk.corpus.stopwords

In [155]:
stp_words.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [158]:
def stop_words(text):
    new_text = []
    for w in text.split():
        if w in stp_words.words('english'):
            new_text.append('')
        else:
            new_text.append(w)
    return ' '.join(new_text)

In [161]:
stop_words('hi this is my name and you can contact me on the given number')

'hi    name    contact    given number'

## Handle Emojis

In [165]:
emoji.demojize('The movie is 😊')

'The movie is :smiling_face_with_smiling_eyes:'

In [166]:
emoji.demojize('Today the weather is very 🔥')

'Today the weather is very :fire:'

In [167]:
emoji.demojize('Why are you so 😠')

'Why are you so :angry_face:'

## Word Tokenize

In [170]:
word_tokenize('This is a very good place.')

['This', 'is', 'a', 'very', 'good', 'place', '.']

In [171]:
word_tokenize('I am going to visit today.')

['I', 'am', 'going', 'to', 'visit', 'today', '.']

In [174]:
word_tokenize('My email id is nsubrat@gmail.com')

['My', 'email', 'id', 'is', 'nsubrat', '@', 'gmail.com']

In [183]:
# using Spacy library

nlp = spacy.load('en_core_web_sm')

In [193]:
doc1 = nlp('You can contact me at nsubrat@gmail.com')
doc2 = nlp('I am going to home!')

In [192]:
for t in doc1:
    print(t)

You
can
contact
me
at
nsubrat@gmail.com


In [194]:
for t in doc2:
    print(t)

I
am
going
to
home
!


## Stemming

In [204]:
ps = PorterStemmer()

def stem_words(text):
    return " ".join([ps.stem(i) for i in text.split()])

In [214]:
sample1 = 'walk walks walking'
sample2 = 'go goes going'

In [215]:
stem_words(sample1)

'walk walk walk'

In [216]:
stem_words(sample2)

'go goe go'

- Here for some words Stemming is not working as expected because stemming converts the words into its root form even if the stem is not a valid word in the Language.
- For that we use Lemmatization which solves this problem.

## Lemmatization

In [219]:
lemma = WordNetLemmatizer()

In [222]:
def lemma_words(text):
    return " ".join([lemma.lemmatize(i,pos='v') for i in text.split()])

In [223]:
sample1 = 'walk walks walking'
sample2 = 'go goes going'

In [225]:
lemma_words(sample1)

'walk walk walk'

In [226]:
lemma_words(sample2)

'go go go'